# Example run: custom scenario

Here we demonstrate how to run with a custom scenario. In order to run this notebook, you will need to add in the code required to run your climate model of choice. See the other example notebooks for how to do this.

## Setup logging

Pyam does its own logging stuff, so we have to configure logging before that import.

In [ ]:
import logging

# Increase the level to reduce the number of log messages
LOGGING_LEVEL = logging.INFO

LOGGER = logging.getLogger("pipeline")
LOGGER.setLevel(LOGGING_LEVEL)
# have to set root logger too to get messages to come through
logging.getLogger().setLevel(LOGGING_LEVEL)

logFormatter = logging.Formatter(
    "%(asctime)s %(name)s %(threadName)s - %(levelname)s:  %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
stdoutHandler = logging.StreamHandler()
stdoutHandler.setFormatter(logFormatter)

logging.getLogger().addHandler(stdoutHandler)

## Other imports

In [ ]:
import os.path
import tempfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas.testing as pdt
import pooch
import pyam

from climate_assessment.cli import run_workflow

## Configuration of input data

### Choice of climate model

See the individual run example notebooks for information about how to get setup with each model.

#### CICERO-SCM

In [ ]:
climate_model_kwargs = dict(
    model="ciceroscm",
    model_version="v2019vCH4",
    probabilistic_file=os.path.join(
        "..",
        "data",
        "cicero",
        "subset_cscm_configfile.json",
    ),
    #     num_cfgs=600,
    num_cfgs=1,
    test_run=False,
)

#### MAGICC

In [ ]:
# Where is the binary?
os.environ["MAGICC_EXECUTABLE_7"] = os.path.join(
    "path", "to", "magicc-v7.5.3", "bin", "magicc"
)

# How many MAGICC workers can run in parallel?
os.environ["MAGICC_WORKER_NUMBER"] = "4"

# Where should the MAGICC workers be located on the filesystem (you need about
# 500Mb space per worker at the moment, they're removed after use)
os.environ["MAGICC_WORKER_ROOT_DIR"] = tempfile.gettempdir()

magicc_data_dir = os.path.join("..", "data", "magicc")

climate_model_kwargs = dict(
    model="magicc",
    model_version="v7.5.3",
    probabilistic_file=os.path.join(
        "path",
        "to",
        "magicc-ar6-0fd0f62-f023edb-drawnset",
        "0fd0f62-derived-metrics-id-f023edb-drawnset.json",
    ),
    num_cfgs=600,
    test_run=False,
)

#### FaIR

In [ ]:
fair_data_dir = os.path.join("..", "data", "fair")

climate_model_kwargs = dict(
    model="fair",
    model_version="1.6.2",
    probabilistic_file=os.path.join(fair_data_dir, "fair-1.6.2-wg3-params-slim.json"),
    fair_extra_config=os.path.join(fair_data_dir, "fair-1.6.2-wg3-params-common.json"),
    num_cfgs=2237,
    test_run=False,
)

### Other config

In [ ]:
# Where should the output be saved?
outdir = os.path.join("..", "data", "output-custom-example-notebook")

# How many scenarios do you want to run in one go?
scenario_batch_size = 20

### Create input emissions

This could be taken from a pre-configured file. Here we create the file on the fly.

In [ ]:
idx = pd.MultiIndex.from_arrays(
    [
        ["stylised"] * 4,
        ["example"] * 4,
        [
            "Emissions|CO2|Energy and Industrial Processes",
            "Emissions|CO2|AFOLU",
            "Emissions|CH4",
            "Emissions|N2O",
        ],
        ["Mt CO2/yr", "Mt CO2/yr", "Mt CH4/yr", "kt N2O/yr"],
        ["World"] * 4,
    ],
    names=["model", "scenario", "variable", "unit", "region"],
)
years = np.arange(2010, 2100 + 1, 5)


def sigmoid(k, x, x0):
    return 1 / (1 + np.exp(-k * (x - x0)))


base = pd.DataFrame(
    data=[
        35000 * sigmoid(-1 / 8, years, 2050),
        4500 * sigmoid(-1 / 8, years, 2050),
        375 / 2 * (1 + sigmoid(-1 / 8, years, 2040)),
        11500 * 2 * sigmoid(-1 / 300, years, 2015),
    ],
    columns=years,
    index=idx,
)
shuffle = base.reset_index()
shuffle["scenario"] = "example_shuffle"
shuffle = shuffle.set_index(base.index.names)
shuffle.loc[
    shuffle.index.get_level_values("variable").isin(
        ["Emissions|CO2|Energy and Industrial Processes"]
    ),
    :,
] = 35000 * sigmoid(-1 / 5, years, 2040)

inp = pyam.IamDataFrame(pd.concat([base, shuffle]))

ax = inp.filter(variable="Emissions|CH4").plot()
ax.set_ylim(ymin=0)
ax = inp.filter(variable="Emissions|N2O").plot()
ax.set_ylim(ymin=0)
ax = inp.filter(variable="Emissions|CO2|Energy and Industrial Processes").plot()
ax.set_ylim(ymin=0)
ax = inp.filter(variable="Emissions|CO2|AFOLU").plot()
ax.set_ylim(ymin=0)

inp

Write file to disk (yes, unfortunately our API currently expects a file on disk, PRs welcome).

In [ ]:
input_emissions_file = "input-emissions.csv"
inp.to_csv(input_emissions_file)

### Choose infiller database file

We run using the infiller database that was used in CMIP6. As a result of the licensing of the scenario data, this file has to be downloaded by hand (see documentation under "Installation", section "Infiller database"). Make sure that the variable `infilling_database_file` points to where you saved this file.

In [ ]:
infilling_database_file = os.path.join(
    "..",
    "data",
    "1652361598937-ar6_emissions_vetted_infillerdatabase_10.5281-zenodo.6390768.csv",
)

## Run the climate assessment workflow

*N.B. the log with information and some warnings will be quite long - but that is nothing to worry about!*

In [ ]:
run_workflow(
    input_emissions_file,
    outdir,
    infilling_database=infilling_database_file,
    scenario_batch_size=scenario_batch_size,
    **climate_model_kwargs,
)

### Load results

*N.B The filename will likely have changed if you have run your own scenarios.*

In [ ]:
output = pyam.IamDataFrame(os.path.join(outdir, "input-emissions_alloutput.xlsx"))
output

### Some basic exploration

Look at the scenario categories.

In [ ]:
output.meta["Category"]

In [ ]:
# A hacky way to examine harmonisation
v = "Emissions|CO2|Energy and Industrial Processes"
v = "Emissions|CO2|AFOLU"
# v = "Emissions|CH4"

ax = output.filter(variable=f"*Infilled|{v}").plot(color="scenario")
inp.filter(variable=v).plot(color="scenario", ax=ax)

In [ ]:
ax = output.filter(variable=f"*Infilled|Emissions|Sulfur").plot(color="scenario")
ax.set_ylim(ymin=0)

Make a plot of median warming.

In [ ]:
ax = output.filter(variable="*|Surface Temperature (GSAT)|*|50.0th Percentile").plot(
    color="scenario"
)
plt.title("Global warming above the 1850-1900 mean")
ax.set_xlim([1995, 2100])

## Conclusion
That's it! You just ran a a full climate assessment workflow going from emissions to temperature (and more) using the functionality from the climate-assessment package, and then visualised the results. 

Naturally, similar workflows can be constructed using CICERO-SCM (on Linux) and MAGICC (on Windows and Linux)!

It is also possible to run from the command line, and build more elaborate workflows. For that, please see the extended documentation.